<a href="https://colab.research.google.com/github/jhtwiz/AI-1-jhtwiz/blob/main/5%EC%A3%BC%EC%B0%A8_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-4로 수능 국어 문제 풀기

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import openai
openai_key = userdata.get('openai')

In [5]:
import json

with open('./2023_11_KICE.json', 'r') as f:
    db = json.load(f)

db[0]

{'id': '2023_11_KICE_1-3',
 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 

데이터 확인 후 일단 잘 돌아가는지 체크하기 위해 4o-mini로 먼저 테스트

In [34]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)


def prediction(problem):
    result = []

    system_prompt = "As a smart student, carefully read the paragraph, questions. Then, think step by step and select the correct answer from the 5 choices. However, at the end, provide only the number of the correct answer (1, 2, 3, 4, or 5) without any explanation."

    for test in problem['problems']:
        question = test['question']
        question_plus = test.get('question_plus', None)
        choices = test['choices']
        answer = test['answer']
        score = test['score']

        user_prompt = f"Paragraph:\n{problem['paragraph']}\n\nQuestion:\n"

        user_prompt += f"{question}\n\n"

        if (question_plus): user_prompt += f"Question Plus:\n{question_plus}\n\n"

        user_prompt += "Choices:\n"

        for i, choice in enumerate(choices):
            user_prompt += f"{i+1} - {choice}\n"

        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )

        pred = completion.choices[0].message.content
        result.append({'pred': pred, 'answer': str(answer), 'score': score})

    return result

In [40]:
def calcAccuracy(results):
    return sum(test['score'] for result in results for test in result if test['pred'] == test['answer'])

In [36]:
results_mini = [prediction(problem) for problem in db]

In [37]:
print(results_mini)

[[{'pred': '2', 'answer': '4', 'score': 2}, {'pred': '4', 'answer': '5', 'score': 3}, {'pred': '1', 'answer': '1', 'score': 2}], [{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 2}, {'pred': '3', 'answer': '3', 'score': 2}, {'pred': '2', 'answer': '2', 'score': 2}, {'pred': '3', 'answer': '5', 'score': 3}, {'pred': '2', 'answer': '2', 'score': 2}], [{'pred': '1', 'answer': '4', 'score': 2}, {'pred': '3', 'answer': '5', 'score': 2}, {'pred': '2', 'answer': '2', 'score': 3}, {'pred': '5', 'answer': '5', 'score': 2}], [{'pred': '3', 'answer': '3', 'score': 2}, {'pred': '3', 'answer': '4', 'score': 2}, {'pred': '3', 'answer': '4', 'score': 2}, {'pred': '3', 'answer': '1', 'score': 3}], [{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '3', 'answer': '3', 'score': 2}, {'pred': '4', 'answer': '3', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 3}], [{'pred': '3', 'answer': '1', 'score': 2}, {'pred': '3', 'answer': '5', 'score': 2}, {'pred': '4', 'answer'

In [41]:
score_mini = calcAccuracy(results_mini)
score_mini

54

mini로 54점이 나왔다. 4o는 어떨까?

In [43]:
#위와 완전 동일하지만 model만 gpt-4o로. 애초에 def prediction(problem, model='gpt-4o')로 할걸 그랬다..
def prediction_4o(problem):
    result = []

    system_prompt = "As a smart student, carefully read the paragraph, questions. Then, think step by step and select the correct answer from the 5 choices. However, at the end, provide only the number of the correct answer (1, 2, 3, 4, or 5) without any explanation."

    for test in problem['problems']:
        question = test['question']
        question_plus = test.get('question_plus', None)
        choices = test['choices']
        answer = test['answer']
        score = test['score']

        user_prompt = f"Paragraph:\n{problem['paragraph']}\n\nQuestion:\n"

        user_prompt += f"{question}\n\n"

        if (question_plus): user_prompt += f"Question Plus:\n{question_plus}\n\n"

        user_prompt += "Choices:\n"

        for i, choice in enumerate(choices):
            user_prompt += f"{i+1} - {choice}\n"

        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )

        pred = completion.choices[0].message.content
        result.append({'pred': pred, 'answer': str(answer), 'score': score})

    return result

In [44]:
results_4o = [prediction_4o(problem) for problem in db]

In [45]:
print(results_4o)

[[{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 3}, {'pred': '1', 'answer': '1', 'score': 2}], [{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 2}, {'pred': '3', 'answer': '3', 'score': 2}, {'pred': '2', 'answer': '2', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 3}, {'pred': '2', 'answer': '2', 'score': 2}], [{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 2}, {'pred': '2', 'answer': '2', 'score': 3}, {'pred': '1', 'answer': '5', 'score': 2}], [{'pred': '3', 'answer': '3', 'score': 2}, {'pred': '5', 'answer': '4', 'score': 2}, {'pred': '4', 'answer': '4', 'score': 2}, {'pred': '1', 'answer': '1', 'score': 3}], [{'pred': '4', 'answer': '4', 'score': 2}, {'pred': '3', 'answer': '3', 'score': 2}, {'pred': '3', 'answer': '3', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 3}], [{'pred': '3', 'answer': '1', 'score': 2}, {'pred': '5', 'answer': '5', 'score': 2}, {'pred': '3', 'answer'

In [46]:
score_4o = calcAccuracy(results_4o)
score_4o

92